# For creating RAG application

1. DATA
2. Embedding Model
3. DATA BASE
4. PROMPT
5. LLM

Connect all the things

1. huggingface hub using hf token(inferencing)
2. load the model in local memory from the hugging face(finetune)

here lots of memory is requierd(7b,10b,50b,100b)

bitandbytes


3. GGML(GPT-Generated Model Language), GGUF(GPT-Generated Unified Format)

4. OLAMAM,LLAMA-CPP,LM-STUDIO

In [55]:
import weaviate
from dotenv import load_dotenv
import os

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

from llama_index.core import ServiceContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import StorageContext

In [56]:
# Load environment variables from .env file
load_dotenv()

# Retrieve the OPENAI_API_KEY
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# print(OPENAI_API_KEY)

# Retrieve the WEAVIATE_API_KEY 
WEAVIATE_API_KEY = os.getenv("WEAVIATE_API_KEY")
# print(WEAVIATE_API_KEY)

# Retrieve the WEAVIATE_CLUSTER link
WEAVIATE_CLUSTER = os.getenv("WEAVIATE_CLUSTER")
# print(WEAVIATE_CLUSTER)


In [42]:
client = weaviate.Client(url= WEAVIATE_CLUSTER, auth_client_secret= weaviate.AuthApiKey(WEAVIATE_API_KEY))

In [7]:
loader=SimpleDirectoryReader(r'D:\iNeuron\GenAI\GenerativeAI\Rag_from_scratch\data')
documents=loader.load_data()

In [9]:
# documents

In [10]:


embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sahil\AppData\Local\llama_index\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
system_prompt=""""<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided

"""

In [13]:
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
# 1. '''system & assistant(LLM)
# 2. user'''

model_name: str = Field(
        default=DEFAULT_HUGGINGFACE_MODEL,
        description=(
            "The model name to use from HuggingFace. "
            "Unused if `model` is passed in directly."
        ),
    )
    context_window: int = Field(
        default=DEFAULT_CONTEXT_WINDOW,
        description="The maximum number of tokens available for input.",
        gt=0,
    )
    max_new_tokens: int = Field(
        default=DEFAULT_NUM_OUTPUTS,
        description="The maximum number of tokens to generate.",
        gt=0,
    )
    system_prompt: str = Field(
        default="",
        description=(
            "The system prompt, containing any extra instructions or context. "
            "The model card on HuggingFace should specify if this is needed."
        ),
    )
    query_wrapper_prompt: PromptTemplate = Field(
        default=PromptTemplate("{query_str}"),
        description=(
            "The query wrapper prompt, containing the query placeholder. "
            "The model card on HuggingFace should specify if this is needed. "
            "Should contain a `{query_str}` placeholder."
        ),
    )
    tokenizer_name: str = Field(
        default=DEFAULT_HUGGINGFACE_MODEL,
        description=(
            "The name of the tokenizer to use from HuggingFace. "
            "Unused if `tokenizer` is passed in directly."
        ),
    )
    device_map: str = Field(
        default="auto", description="The device_map to use. Defaults to 'auto'."
    )
    stopping_ids: List[int] = Field(
        default_factory=list,
        description=(
            "The stopping ids to use. "
            "Generation stops when these token IDs are predicted."
        ),
    )
    tokenizer_outputs_to_remove: list = Field(
        default_factory=list,
        description=(
            "The outputs to remove from the tokenizer. "
            "Sometimes huggingface tokenizers return extra inputs that cause errors."
        ),
    )
    tokenizer_kwargs: dict = Field(
        default_factory=dict, description="The kwargs to pass to the tokenizer."
    )
    model_kwargs: dict = Field(
        default_factory=dict,
        description="The kwargs to pass to the model during initialization.",
    )
    generate_kwargs: dict = Field(
        default_factory=dict,
        description="The kwargs to pass to the model during generation.",
    )
    is_chat_model: bool = Field(
        default=False,
        description=(
            LLMMetadata.__fields__["is_chat_model"].field_info.description
            + " Be sure to verify that you either pass an appropriate tokenizer "
            "that can convert prompts to properly formatted chat messages or a "
            "`messages_to_prompt` that does so."
        ),
    )

# Local store

In [16]:
import torch
llm=HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}

)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sahil\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [17]:


Service_Context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

C:\Users\sahil\AppData\Local\Temp\ipykernel_40964\878835869.py:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  Service_Context=ServiceContext.from_defaults(


In [23]:
index = VectorStoreIndex.from_documents(documents=documents, service_context=Service_Context)
query_engine =  index.as_query_engine(k=1)

In [24]:
question = "which matrics we can use for evaluating the machine translation task?"
result = query_engine.query(question)
result.response

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\transformers\generation\configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'We can use the matrix given in Figure 3 to evaluate the machine translation task. This matrix represents the weights, αij, of the annotation of the j-th source word for the i-th translated word. The matrix can be used to investigate the soft alignment between the translated sentence from the model and the input sentence. The matrix can also be used to visualize and see which word from the input sentence were considered more important for generating the target word. Additionally, we can use the BLEU score to evaluate the machine translation task. The BLEU score is a widely used metric for evaluating machine translation models. It measures the similarity between the generated text and a set of human reference texts. The BLEU score is calculated based on the number of n-grams (unigrams, bigrams, trigrams, etc.) that the generated text shares with the reference texts. The higher the BLEU score, the better the machine translation model.'

In [25]:
result

Response(response='We can use the matrix given in Figure 3 to evaluate the machine translation task. This matrix represents the weights, αij, of the annotation of the j-th source word for the i-th translated word. The matrix can be used to investigate the soft alignment between the translated sentence from the model and the input sentence. The matrix can also be used to visualize and see which word from the input sentence were considered more important for generating the target word. Additionally, we can use the BLEU score to evaluate the machine translation task. The BLEU score is a widely used metric for evaluating machine translation models. It measures the similarity between the generated text and a set of human reference texts. The BLEU score is calculated based on the number of n-grams (unigrams, bigrams, trigrams, etc.) that the generated text shares with the reference texts. The higher the BLEU score, the better the machine translation model.', source_nodes=[NodeWithScore(node=

In [26]:
print(result)

We can use the matrix given in Figure 3 to evaluate the machine translation task. This matrix represents the weights, αij, of the annotation of the j-th source word for the i-th translated word. The matrix can be used to investigate the soft alignment between the translated sentence from the model and the input sentence. The matrix can also be used to visualize and see which word from the input sentence were considered more important for generating the target word. Additionally, we can use the BLEU score to evaluate the machine translation task. The BLEU score is a widely used metric for evaluating machine translation models. It measures the similarity between the generated text and a set of human reference texts. The BLEU score is calculated based on the number of n-grams (unigrams, bigrams, trigrams, etc.) that the generated text shares with the reference texts. The higher the BLEU score, the better the machine translation model.


In [27]:
result2=query_engine.query("what is attention mechnisam?")
print(result2)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Attention is a mechanism used in machine translation to retain and utilize all the hidden state of the input sequence during the decoding phase. It allows the model to focus and pay more attention to the relevant part of the input sequence, which helps to improve the translation accuracy. The attention mechanism works by creating an alignment between each time step of the decoder output and all of the encoder hidden state. This alignment is learned during the training process, and each output of the decoder can selectively pick out specific elements from the sequence to produce the output. The attention mechanism helps to address the bottle-neck problem in RNN Encoder-Decoder models, where the complete sequence of information of the source sentence must be captured by one single vector, which is difficult for the decoder to summarize large input sequences at once.


In [28]:
result2.response

'Attention is a mechanism used in machine translation to retain and utilize all the hidden state of the input sequence during the decoding phase. It allows the model to focus and pay more attention to the relevant part of the input sequence, which helps to improve the translation accuracy. The attention mechanism works by creating an alignment between each time step of the decoder output and all of the encoder hidden state. This alignment is learned during the training process, and each output of the decoder can selectively pick out specific elements from the sequence to produce the output. The attention mechanism helps to address the bottle-neck problem in RNN Encoder-Decoder models, where the complete sequence of information of the source sentence must be captured by one single vector, which is difficult for the decoder to summarize large input sequences at once.'

In [31]:
from IPython.display import Markdown, display
# fixing unicode error in google colab
import locale

In [30]:
display(Markdown(f"<b>{result}</b>"))

<b>We can use the matrix given in Figure 3 to evaluate the machine translation task. This matrix represents the weights, αij, of the annotation of the j-th source word for the i-th translated word. The matrix can be used to investigate the soft alignment between the translated sentence from the model and the input sentence. The matrix can also be used to visualize and see which word from the input sentence were considered more important for generating the target word. Additionally, we can use the BLEU score to evaluate the machine translation task. The BLEU score is a widely used metric for evaluating machine translation models. It measures the similarity between the generated text and a set of human reference texts. The BLEU score is calculated based on the number of n-grams (unigrams, bigrams, trigrams, etc.) that the generated text shares with the reference texts. The higher the BLEU score, the better the machine translation model.</b>

---
# This is using Weaviate cloud which support linux

In [32]:
locale.getpreferredencoding = lambda: "UTF-8"

In [43]:
client=weaviate.Client(url=WEAVIATE_CLUSTER,auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY))
vector_store=WeaviateVectorStore(weaviate_client=client,index_name="Llamaindex")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

AttributeError: 'Client' object has no attribute 'collections'

In [60]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context,embed_model=embed_model
)

NameError: name 'storage_context' is not defined

In [61]:
query_engine = index.as_query_engine(llm=llm)

In [62]:
result3=query_engine.query("what is attention?")

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\transformers\generation\configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [63]:
result3.response

'Attention is a mechanism used in machine translation to retain and utilize all the hidden state of the input sentence during the decoding phase. It allows the model to focus and pay more attention to the relevant part of the input sequence. During the decoding phase, the model creates an alignment between each time step of the decoder output and all of the encoder hidden state. This allows the model to selectively pick out specific elements from the sequence to produce the output. The attention mechanism helps to memorize this information for longer sentences.'